The purpose of this code is to quantify the nuclei, the size and number of synuclein inclusions, and the lysosomal surface area in the lysotracker experiments.

Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.filters import gaussian, threshold_otsu
from skimage.morphology import remove_small_objects, binary_dilation, disk, binary_closing
from skimage.measure import label, regionprops
from skimage import exposure

Define Sub Functions

In [ ]:
def extract_image_paths(folder):
    """Extract all image file paths from the specified folder."""
    return [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

def read_image(image_path):
    """Read the LSM image from the specified path."""
    return imread(image_path)

def extract_channels(image):
    """Extract DAPI, green, and red channels from the image."""
    return image[0], image[1], image[2]

def otsu_thresholding(channel):
    """Apply Otsu's thresholding and morphological closing to the channel."""
    threshold_value = threshold_otsu(channel)
    binary_image = channel > threshold_value
    closed_image = binary_closing(binary_image, disk(3))
    labeled_image = label(closed_image)
    return labeled_image

def calculate_surface_area(labeled_image, channel):
    """Calculate the total surface area for labeled regions."""
    props = regionprops(labeled_image, channel)
    return sum(prop.area for prop in props)

def preprocess_dapi_channel(dapi_channel):
    """Preprocess the DAPI channel for nuclei quantification."""
    blurred_dapi = gaussian(dapi_channel, sigma=2)
    threshold_value = threshold_otsu(blurred_dapi)
    binary_image = blurred_dapi > threshold_value
    cleaned_image = remove_small_objects(binary_image, min_size=400)
    merged_image = binary_dilation(cleaned_image, footprint=disk(5))
    labeled_image = label(merged_image)
    return labeled_image

def count_nuclei(labeled_image):
    """Count the number of nuclei in the labeled image."""
    return len(np.unique(labeled_image)) - 1

def preprocess_green_channel(green_channel):
    """Preprocess the green channel for inclusion quantification."""
    confocal_img = exposure.adjust_sigmoid(green_channel, cutoff=0.25)
    confocal_img = (confocal_img - confocal_img.min()) / (confocal_img.max() - confocal_img.min())
    return confocal_img

def threshold_inclusions(confocal_img):
    """Threshold the preprocessed green channel to segment inclusions."""
    binary_image = confocal_img > 0.6
    labeled_image = label(binary_image)
    return labeled_image

def measure_inclusion_sizes(labeled_image, confocal_img):
    """Measure the sizes of inclusions."""
    props = regionprops(labeled_image, confocal_img)
    sizes = [prop.area for prop in props]
    return [size for size in sizes if size > 10]

def add_to_dataframe(sizes_df_new, sizes, path):
    """Add sizes of inclusions to the DataFrame."""
    sizes_df_add = pd.DataFrame(sizes, columns=[f'image {path}'])
    if sizes_df_new is None:
        return sizes_df_add
    else:
        return pd.concat([sizes_df_new, sizes_df_add], axis=1)

def calculate_metrics(surface_areas, number_of_nuclei_list, mean_sizes_of_inclusions, sizes_df_new):
    """Calculate additional metrics and return the second DataFrame."""
    sizes_df_new_nuclei = sizes_df_new.transpose()
    number_of_inclusions = sizes_df_new_nuclei.count(axis=1)
    average_number_of_inclusions = number_of_inclusions / np.array(number_of_nuclei_list)
    surface_area_to_nuclei = [sa / n for sa, n in zip(surface_areas, number_of_nuclei_list)]

    return pd.DataFrame({
        "number_of_inclusions": number_of_inclusions,
        "Number_of_Nuclei": number_of_nuclei_list,
        "Average_Number_of_Inclusions_per_Cell": average_number_of_inclusions,
        "Surface_areas_lysosomes": surface_areas,
        "Surface_areas_to_nuclei": surface_area_to_nuclei,
        "mean_sizes_of_inclusions": mean_sizes_of_inclusions
    })

def save_to_excel(sizes_df_new, excel_2):
    """Save the DataFrames to Excel files."""
    excel_2.to_excel("SUMMARY.xlsx")

Define Main Function

In [ ]:
def main(image_folder):
    # Initialize lists to store results
    surface_areas = []
    number_of_nuclei_list = []
    mean_sizes_of_inclusions = []
    sizes_df_new = None

    images_to_analyze = extract_image_paths(image_folder)

    # Iterate over each image
    for path in images_to_analyze:
        image = read_image(path)

        dapi_channel, green_channel, red_channel = extract_channels(image)
        
        # Process red channel for surface area
        labeled_image_otsu = otsu_thresholding(red_channel)
        surface_area = calculate_surface_area(labeled_image_otsu, red_channel)
        surface_areas.append(surface_area)

        # Process DAPI channel for nuclei counting
        labeled_image_dapi = preprocess_dapi_channel(dapi_channel)
        n_nuclei = count_nuclei(labeled_image_dapi)
        number_of_nuclei_list.append(n_nuclei)

        # Process green channel for inclusion quantification
        confocal_img = preprocess_green_channel(green_channel)
        labeled_image_inclusions = threshold_inclusions(confocal_img)
        inclusion_sizes = measure_inclusion_sizes(labeled_image_inclusions, confocal_img)
        mean_sizes_of_inclusions.append(np.mean(inclusion_sizes))

        # Add to DataFrame
        sizes_df_new = add_to_dataframe(sizes_df_new, inclusion_sizes, os.path.basename(path))

    # Calculate metrics and save results
    excel_2 = calculate_metrics(surface_areas, number_of_nuclei_list, mean_sizes_of_inclusions, sizes_df_new)
    save_to_excel(sizes_df_new, excel_2)

if __name__ == "__main__":
    image_folder = 'test_images'
    main(image_folder)